## Newton para Sistemas não Lineares

O método de Newton para sistemas não lineares busca encontrar a solução de um sistema f(x) = 0 utilizando aproximações sucessivas e a inversa do Jacobiano.


In [1]:
import numpy as np
import sympy as sp
from tabulate import tabulate

In [12]:
def newton_sistemas(funcs, vars, x0, tol, max_iter=100):
    """
    Resolve sistemas de equações não lineares pelo método de Newton.

    Parâmetros:
    funcs -- Lista de funções do sistema (expressões simbólicas)
    vars -- Lista de variáveis simbólicas
    x0 -- Aproximação inicial (array ou lista)
    tol -- Erro de tolerância
    max_iter -- Número máximo de iterações

    Retorna:
    Solução aproximada e tabela com as iterações
    """
    # Calcula o Jacobiano simbólico
    J = sp.Matrix([[sp.diff(f, var) for var in vars] for f in funcs])
    
    # Converte as funções e o Jacobiano para funções numéricas
    f_lambdify = sp.lambdify(vars, funcs, 'numpy')
    J_lambdify = sp.lambdify(vars, J, 'numpy')

    # Inicializa a solução inicial
    x = np.array(x0, dtype=float)
    tabela = []

    for iter in range(max_iter):
        # Avalia as funções e o Jacobiano no ponto atual
        F_val = np.array(f_lambdify(*x), dtype=float).flatten()
        J_val = np.array(J_lambdify(*x), dtype=float)

        # Verifica se o Jacobiano está mal condicionado
        if np.linalg.cond(J_val) > 1e12:
            print("Jacobiano mal condicionado! Pode não haver convergência.")
            return None, tabela

        try:
            # Resolve o sistema linear J * delta_x = -F
            delta_x = np.linalg.solve(J_val, -F_val)
        except np.linalg.LinAlgError:
            print("Erro: Jacobiano singular ou indefinido.")
            return None, tabela

        # Atualiza a solução
        x_new = x + delta_x

        # Calcula os erros absoluto e relativo
        erro_absoluto = np.linalg.norm(delta_x)
        erro_relativo = erro_absoluto / (np.linalg.norm(x_new) if np.linalg.norm(x_new) != 0 else 1)

        # Armazena os resultados na tabela
        tabela.append([iter, *x, erro_absoluto, erro_relativo])

        # Verifica a condição de parada
        if erro_absoluto <= tol and iter > 10:
            return np.round(x_new, 4), tabela

        # Atualiza o ponto para a próxima iteração
        x = x_new

    print("Número máximo de iterações atingido.")
    return np.round(x, 4), tabela

In [10]:
funcs = [sp.sympify("x1**2+x2**2-4"), sp.sympify("exp(x1)+x2-1")]

In [13]:
# Entrada do usuário
if __name__ == "__main__":
    n = int(input("Digite o número de equações/variáveis: "))
    vars = sp.symbols(f'x1:{n+1}')
    
    # Define as funções do sistema
    # print("Digite as funções do sistema:")
    # funcs = []
    # for i in range(n):
    #     func_input = input(f"f{i+1}(x1, ..., xn) = ")
    #     funcs.append(sp.sympify(func_input))

    # Define a aproximação inicial
    x0 = [float(input(f"Digite x{i+1} inicial: ")) for i in range(n)]
    tol = float(input("Digite o erro de tolerância: "))

    # Executa o método de Newton
    resultado = newton_sistemas(funcs, vars, x0, tol)

    if resultado is None:
        print("O método não convergiu ou houve um erro numérico.")
    else:
        solucao, tabela = resultado
        
        # Exibe a tabela de resultados
        headers = ["Iteração"] + [f"x{i+1}" for i in range(n)] + ["Erro Absoluto", "Erro Relativo"]
        print(tabulate(tabela, headers=headers, floatfmt=".4f"))
        
        print(f"Solução aproximada: {solucao}")

  Iteração       x1      x2    Erro Absoluto    Erro Relativo
----------  -------  ------  ---------------  ---------------
         0   1.0000  1.7183           1.8757           0.6841
         1  -0.6151  2.6721           2.3079           0.7557
         2  -2.6316  1.5495           0.9099           0.4141
         3  -2.0118  0.8834           0.1919           0.0955
         4  -1.8246  0.8412           0.0092           0.0046
         5  -1.8163  0.8374           0.0000           0.0000
         6  -1.8163  0.8374           0.0000           0.0000
         7  -1.8163  0.8374           0.0000           0.0000
         8  -1.8163  0.8374           0.0000           0.0000
         9  -1.8163  0.8374           0.0000           0.0000
        10  -1.8163  0.8374           0.0000           0.0000
        11  -1.8163  0.8374           0.0000           0.0000
Solução aproximada: [-1.8163  0.8374]
